Trying to follow this tutorial
https://www.ai-contentlab.com/2023/03/how-to-implement-timesformer-for-video.html

In [17]:
from transformers import TimesformerConfig, TimesformerModel, AutoImageProcessor

In [38]:
import torch
import os
import cv2
import random
import numpy as np

In [39]:
IMG_SIZE = 224

In [64]:
image_processor = AutoImageProcessor.from_pretrained("MCG-NJU/videomae-base")
model = TimesformerModel.from_pretrained("facebook/timesformer-base-finetuned-k400", ignore_mismatched_sizes=True, num_labels=5)

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
Some weights of the model checkpoint at facebook/timesformer-base-finetuned-k400 were not used when initializing TimesformerModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing TimesformerModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TimesformerModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [77]:
configuration = TimesformerConfig()
model = TimesformerModel(configuration)

In [78]:
def frames_from_video_file(video_path, MAX_SEQ_LENGTH, frame_step, output_size = (IMG_SIZE, IMG_SIZE)):
    result = []
  
    src = cv2.VideoCapture(str(video_path))  

    video_length = src.get(cv2.CAP_PROP_FRAME_COUNT)

    need_length = 1 + (MAX_SEQ_LENGTH - 1) * frame_step

    if need_length > video_length:
        start = 0
    else:
        max_start = video_length - need_length
        start = random.randint(0, max_start + 1)

    src.set(cv2.CAP_PROP_POS_FRAMES, start)
    # ret is a boolean indicating whether read was successful, frame is the image itself
    ret, frame = src.read()
    frame = cv2.resize(frame, output_size)
    cv2.normalize(frame, frame, 0, 255, cv2.NORM_MINMAX)
    #frame = frame.transpose(2,1,0)
    #frame = torch.tensor(frame)
    #frame = frame.to(torch.float32)
    result.append(frame)

    for _ in range(MAX_SEQ_LENGTH - 1):
        for _ in range(frame_step):
            ret, frame = src.read()
        if ret:
            frame = cv2.resize(frame, output_size)
            cv2.normalize(frame, frame, 0, 255, cv2.NORM_MINMAX)
            #frame = frame.transpose(2,1,0)
            #frame = torch.tensor(frame)
            #frame = frame.to(torch.float32)
            result.append(frame)
        else:
            frame = np.zeros_like(result[0])
            #frame = torch.tensor(frame)
            #frame = frame.to(torch.float32)
            result.append(frame)
    src.release()
    result = np.array(result)[..., [2, 1, 0]]

    return result

In [79]:
class VideoDataset(torch.utils.data.Dataset):
    def __init__(self, df, root_dir, MAX_SEQ_LENGTH, frame_step):
        self.video_paths = df["video-name"].values.tolist()
        self.labels = df["label"].values.tolist()
        self.n_frames = MAX_SEQ_LENGTH
        self.root_dir = root_dir
        self.frame_step = frame_step
        
    def __len__(self):
        return len(self.video_paths)
    
    def __getitem__(self, idx):
        path = self.video_paths[idx]
        label = self.labels[idx]
        frames = frames_from_video_file(os.path.join(self.root_dir, path), self.n_frames, self.frame_step)
        frames = torch.stack(frames)
        return frames, label

In [80]:
import pandas as pd

In [81]:
df = pd.read_csv("../data/mirror-data.csv")
df["Action"] = df["Action"].str.rstrip()
df = df[df.Action != "Talking&Yawning"]
df["label"] = df.Action.astype('category').cat.codes

In [82]:
i = 0
dfTrain = pd.DataFrame()
dfTest = pd.DataFrame()

while i<len(df):
    if i%5==0:
        dfTest = pd.concat([dfTest, df.iloc[[i]]])
    else :
        dfTrain = pd.concat([dfTrain, df.iloc[[i]]])
    i+=1

In [83]:
MAX_SEQ_LENGTH = 8
frame_step = 1

In [84]:
train_ds = VideoDataset(dfTrain, "../data/YawDD/YawDD dataset/Mirror/all/", MAX_SEQ_LENGTH, frame_step)
test_ds = VideoDataset(dfTest, "../data/YawDD/YawDD dataset/Mirror/all/", MAX_SEQ_LENGTH, frame_step)

In [85]:
from torch.utils.data import DataLoader

In [86]:
train_loader = DataLoader(train_ds, batch_size=4)
val_loader = DataLoader(test_ds, batch_size=4)

In [ ]:
train_loader

In [87]:
import torch.optim as optim
from torch import nn
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [88]:
num_epochs = 2

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
    # Evaluate on validation set
    correct = 0
    total = 0
    with torch.no_grad():
        for data in val_loader:
            inputs, labels = data
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    val_acc = 100 * correct / total
    
    print('[%d] loss: %.3f, val_acc: %.3f' %
          (epoch + 1, running_loss / len(train_loader), val_acc))

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:5                                                                                    │
│                                                                                                  │
│    2                                                                                             │
│    3 for epoch in range(num_epochs):                                                             │
│    4 │   running_loss = 0.0                                                                      │
│ ❱  5 │   for i, data in enumerate(train_loader, 0):                                              │
│    6 │   │   inputs, labels = data                                                               │
│    7 │   │   optimizer.zero_grad()                                                               │
│    8 │   │   outputs = model(inputs)                                                             │
│                                                                                                  │
│ c:\Users\xuton\Documents\Work\Year                                                               │
│ 5\Capstone\Code\se4450-project-group-23\model\.venv\lib\site-packages\torch\utils\data\dataloade │
│ r.py:634 in __next__                                                                             │
│                                                                                                  │
│    631 │   │   │   if self._sampler_iter is None:                                                │
│    632 │   │   │   │   # TODO(https://github.com/pytorch/pytorch/issues/76750)                   │
│    633 │   │   │   │   self._reset()  # type: ignore[call-arg]                                   │
│ ❱  634 │   │   │   data = self._next_data()                                                      │
│    635 │   │   │   self._num_yielded += 1                                                        │
│    636 │   │   │   if self._dataset_kind == _DatasetKind.Iterable and \                          │
│    637 │   │   │   │   │   self._IterableDataset_len_called is not None and \                    │
│                                                                                                  │
│ c:\Users\xuton\Documents\Work\Year                                                               │
│ 5\Capstone\Code\se4450-project-group-23\model\.venv\lib\site-packages\torch\utils\data\dataloade │
│ r.py:678 in _next_data                                                                           │
│                                                                                                  │
│    675 │                                                                                         │
│    676 │   def _next_data(self):                                                                 │
│    677 │   │   index = self._next_index()  # may raise StopIteration                             │
│ ❱  678 │   │   data = self._dataset_fetcher.fetch(index)  # may raise StopIteration              │
│    679 │   │   if self._pin_memory:                                                              │
│    680 │   │   │   data = _utils.pin_memory.pin_memory(data, self._pin_memory_device)            │
│    681 │   │   return data                                                                       │
│                                                                                                  │
│ c:\Users\xuton\Documents\Work\Year                                                               │
│ 5\Capstone\Code\se4450-project-group-23\model\.venv\lib\site-packages\torch\utils\data\_utils\fe │
│ tch.py:51 in fetch                                                                               │
│                                                                                                  │
│   48 │   │   │   if hasattr(self.dataset, "__getitems__") a